In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np 
import pandas as pd 
import scipy as sp
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
from pykrige.ok import OrdinaryKriging


In [2]:
deposit_data = pd.read_csv("../../Curated_data/final_dataset.csv", low_memory=False)


In [12]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'deposit_data' is your DataFrame
# Replace this line with your actual data loading code
# deposit_data = pd.read_csv('your_data_file.csv')

# Extract X, Y, and Z columns from deposit_data
X = deposit_data['X']
Y = deposit_data['Y']
Z = deposit_data['Z']

# Create a 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(x=X, y=Y, z=Z, mode='markers')])

# Update layout
fig.update_layout(scene=dict(aspectmode='data'))

# Set axis labels
fig.update_layout(scene=dict(xaxis=dict(title='X-axis'),
                             yaxis=dict(title='Y-axis'),
                             zaxis=dict(title='Z-axis')))

# Set plot title
fig.update_layout(title='3D Scatter Plot of deposit_data')

# Show the plot
fig.show()


In [14]:
print(X.min(),Y.min(),Z.min() ,X.max(),Y.max(),Z.max())

0.0 0.0 0.0 0.8667650393188396 1.0 0.5017860034908529


1.4628216990943845


In [8]:
1/d

0.3333333333333333

In [48]:
total_columns = ['CP_Total','PO_Total', 'PY_Total']

#all covariates
covariates = total_columns[:3] + ['RQD_Pct', 'Cr_ppm'] 

In [49]:
# Assuming deposit_data is your DataFrame
# Extract the names of the first 98 columns
phi_columns = deposit_data.columns[10:].tolist()
phi_columns[-1]


'phi_6402'

In [50]:
# Function to print evaluation metrics
def print_metrics(actual, predicted, set_name, num_predictors):
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actual, predicted)
    r2 = r2_score(actual, predicted)

    n = len(actual)
    adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - num_predictors - 1))

    print(f"Metrics for {set_name} set:")
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  R^2: {r2:.4f}")
    print(f"  Adjusted R^2: {adjusted_r2:.4f}\n")


# Deepkriging covariates

In [ ]:
phi_columns = deposit_data.columns[10:].tolist()
p = len(covariates) +len(phi_columns)

torch.manual_seed(42)
np.random.seed(42)

# Assuming deposit_data, covariates, and other necessary variables are defined

# Create an array to store metrics for each fold
test_mse_list = []
test_rmse_list = []
test_mae_list = []
test_r2_list = []
test_adjusted_r2_list = []


# Define the number of folds for cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Perform k-fold cross-validation
for fold, (train_index, test_index) in enumerate(kf.split(deposit_data)):
    train_data, test_data = deposit_data.iloc[train_index], deposit_data.iloc[test_index]

    x_train = train_data[phi_columns + covariates].values
    y_train = train_data['Density_gcm3'].values

    x_test = test_data[phi_columns + covariates].values
    y_test = test_data['Density_gcm3'].values

    # Define your neural network
    model = nn.Sequential(
        nn.Linear(in_features=p, out_features=100),
        nn.ReLU(),
        nn.Dropout(0.5) ,
        nn.BatchNorm1d(100),
        nn.Linear(in_features=100, out_features=100),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(in_features=100, out_features=100),
        nn.ReLU(),
        nn.BatchNorm1d(100),
        nn.Linear(in_features=100, out_features=1))


    mse_loss = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.005)

    train_losses = []  # To store training losses during training
    test_losses = []   # To store test losses during training

    # Training loop
    for step in range(601):
        pre = model(torch.tensor(x_train, dtype=torch.float32))
        mse = mse_loss(pre, torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32))
        cost = mse

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        pre_test = model(torch.tensor(x_test, dtype=torch.float32))
        mse_test = mse_loss(pre_test, torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32))
        test_losses.append(mse_test.item())

    # Store metrics for this fold
    test_predictions_fold = model(torch.tensor(x_test, dtype=torch.float32)).detach().numpy().flatten()
    test_mse_list.append(mean_squared_error(y_test, test_predictions_fold))
    test_mae_list.append(mean_absolute_error(y_test, test_predictions_fold))
    test_r2_list.append(r2_score(y_test, test_predictions_fold))

     # Calculate adjusted R-squared
    n = len(y_test)
    sst = np.sum((y_test - np.mean(y_test)) ** 2)
    ssr = np.sum((test_predictions_fold - y_test) ** 2)
    r2 = 1 - (ssr / sst)
    adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - p - 1))
    test_adjusted_r2_list.append(adjusted_r2)

   # Print metrics for the current fold
    print(f"\nMetrics for Fold {fold + 1}:")
    print_metrics(y_test, test_predictions_fold, "Test", p)

# Print average metrics across folds
print("\nAverage Metrics Across Folds:")
print(f"  Average MSE: {np.mean(test_mse_list):.4f}")
print(f"  Average MAE: {np.mean(test_mae_list):.4f}")
print(f"  Average Adjusted R2: {np.mean(test_adjusted_r2_list):.4f}")
print(f"  STD MSE: {np.std(test_mse_list):.4f}")
print(f"  STD MAE: {np.std(test_mae_list):.4f}")
print(f"  STD Adjusted R2: {np.std(test_adjusted_r2_list):.4f}")


Metrics for Fold 1:
Metrics for Test set:
  MSE: 0.0024
  RMSE: 0.0485
  MAE: 0.0355
  R^2: 0.8524
  Adjusted R^2: 1.0063


Metrics for Fold 2:
Metrics for Test set:
  MSE: 0.0031
  RMSE: 0.0559
  MAE: 0.0372
  R^2: 0.7618
  Adjusted R^2: 1.0101


Metrics for Fold 3:
Metrics for Test set:
  MSE: 0.0029
  RMSE: 0.0540
  MAE: 0.0384
  R^2: 0.8020
  Adjusted R^2: 1.0084


Metrics for Fold 4:
Metrics for Test set:
  MSE: 0.0018
  RMSE: 0.0427
  MAE: 0.0312
  R^2: 0.8648
  Adjusted R^2: 1.0057


Metrics for Fold 5:
Metrics for Test set:
  MSE: 0.0030
  RMSE: 0.0552
  MAE: 0.0393
  R^2: 0.8306
  Adjusted R^2: 1.0072


Metrics for Fold 6:
Metrics for Test set:
  MSE: 0.0028
  RMSE: 0.0525
  MAE: 0.0367
  R^2: 0.7834
  Adjusted R^2: 1.0092


Metrics for Fold 7:
Metrics for Test set:
  MSE: 0.0025
  RMSE: 0.0505
  MAE: 0.0383
  R^2: 0.8590
  Adjusted R^2: 1.0060


Metrics for Fold 8:
Metrics for Test set:
  MSE: 0.0021
  RMSE: 0.0456
  MAE: 0.0338
  R^2: 0.8844
  Adjusted R^2: 1.0049


Metrics

# Deepkriging no covariates

In [ ]:
torch.manual_seed(42)
np.random.seed(42)

phi_columns = deposit_data.columns[10:].tolist()
p = len(phi_columns)

# Create an array to store metrics for each fold
test_mse_list = []
test_rmse_list = []
test_mae_list = []
test_r2_list = []
test_adjusted_r2_list = []


# Define the number of folds for cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Perform k-fold cross-validation
for fold, (train_index, test_index) in enumerate(kf.split(deposit_data)):
    train_data, test_data = deposit_data.iloc[train_index], deposit_data.iloc[test_index]

    x_train = train_data[phi_columns].values
    y_train = train_data['Density_gcm3'].values

    x_test = test_data[phi_columns].values
    y_test = test_data['Density_gcm3'].values

    # Define your neural network
    model = nn.Sequential(
        nn.Linear(in_features=p, out_features=100),
        nn.ReLU(),
        nn.Dropout(0.5) ,
        nn.BatchNorm1d(100),
        nn.Linear(in_features=100, out_features=100),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(in_features=100, out_features=100),
        nn.ReLU(),
        nn.BatchNorm1d(100),
        nn.Linear(in_features=100, out_features=1))


    mse_loss = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.005)

    train_losses = []  # To store training losses during training
    test_losses = []   # To store test losses during training

    # Training loop
    for step in range(601):
        pre = model(torch.tensor(x_train, dtype=torch.float32))
        mse = mse_loss(pre, torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32))
        cost = mse

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        pre_test = model(torch.tensor(x_test, dtype=torch.float32))
        mse_test = mse_loss(pre_test, torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32))
        test_losses.append(mse_test.item())

    # Store metrics for this fold
    test_predictions_fold = model(torch.tensor(x_test, dtype=torch.float32)).detach().numpy().flatten()
    test_mse_list.append(mean_squared_error(y_test, test_predictions_fold))
    test_mae_list.append(mean_absolute_error(y_test, test_predictions_fold))
    test_r2_list.append(r2_score(y_test, test_predictions_fold))

     # Calculate adjusted R-squared
    n = len(y_test)
    sst = np.sum((y_test - np.mean(y_test)) ** 2)
    ssr = np.sum((test_predictions_fold - y_test) ** 2)
    r2 = 1 - (ssr / sst)
    adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - p - 1))
    test_adjusted_r2_list.append(adjusted_r2)

   # Print metrics for the current fold
    print(f"\nMetrics for Fold {fold + 1}:")
    print_metrics(y_test, test_predictions_fold, "Test", p)

# Print average metrics across folds
print("\nAverage Metrics Across Folds:")
print(f"  Average MSE: {np.mean(test_mse_list):.4f}")
print(f"  Average MAE: {np.mean(test_mae_list):.4f}")
print(f"  Average Adjusted R2: {np.mean(test_adjusted_r2_list):.4f}")
print(f"  STD MSE: {np.std(test_mse_list):.4f}")
print(f"  STD MAE: {np.std(test_mae_list):.4f}")
print(f"  STD Adjusted R2: {np.std(test_adjusted_r2_list):.4f}")


Metrics for Fold 1:
Metrics for Test set:
  MSE: 0.0026
  RMSE: 0.0511
  MAE: 0.0373
  R^2: 0.8365
  Adjusted R^2: 1.0069



KeyboardInterrupt: 